#Reinforce

  ![3-1](https://drive.google.com/uc?id=11K3afpbihNRu4MtW4JbU3U4BBXheeoZO)

In [0]:
!pip install --upgrade gym
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1


!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

!wget -O utils.py https://drive.google.com/uc?id=18arMwKmz1KUpNxxD-pMPDtWSm8I2OG0i


import gym
import utils
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

from collections import namedtuple


import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

import torch
import torch.autograd as autograd
import torch.nn as nn
from enum import Enum

import copy


Requirement already up-to-date: gym in /usr/local/lib/python3.6/dist-packages (0.14.0)
Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (41.2.0)
--2019-10-03 02:47:48--  https://drive.google.com/uc?id=18arMwKmz1KUpNxxD-pMPDtWSm8I2OG0i
Resolving drive.google.com (drive.google.com)... 173.194.203.139, 173.194.203.113, 173.194.203.138, ...
Connecting to drive.google.com (drive.google.com)|173.194.203.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-0k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/osbhjv1529gi5kcvungicud31vrv9ies/1570068000000/12953947379101213852/*/18arMwKmz1KUpNxxD-pMPDtWSm8I2OG0i [following]
--2019-10-03 02:47:48--  https://doc-10-0k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/osbhjv1529gi5kcvungicud31vrv9ies/1570068000000/12953947379101213852/*/18arMwKmz1KUpNxxD-pMPDtWSm8I2OG0i
Resolving doc-10-0k-docs.googleusercont

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


In [0]:
class Vanilla_reinforce_agent():

  def __init__(self, input_size, output_size):
    self.w = np.random.rand(input_size[0], output_size)
    self.actions = range(output_size)


  def train(self, env, num_episodes, max_steps, gamma = 0.9, lr = 0.000025):
    episode_rewards = []
    for e in range(num_episodes):
      state = env.reset()
      episode_reward = 0
      rewards = []
      grads = []
      for step in range(max_steps):
        probs = self.policy(state)
        action = np.random.choice(self.actions, p=probs)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -10

        rewards.append(reward)
        grad = self.p_gradient(state, probs, action)
        grads.append(grad)
        episode_reward += reward

        state = next_state

        if done or step == max_steps-1:
          episode_rewards.append(episode_reward)
          print("Episode " + str(e) + ": " + str(episode_reward))
          break


      for i in range(len(grads)):
        grad = grads[i] * lr
        future_discounted_rewards = sum([ r * (gamma ** r) for t,r in enumerate(rewards[i:])])
        self.w += grad*future_discounted_rewards

  def policy(self, state):
    # forward propagate state trough the Neural network
    h = state.dot(self.w) # dot prodcut state and w
    o = np.exp(h)  # softmax activation function
    0 = o/np.sum(o)
    return o

  # Vectorized softmax Jacobian
  def softmax_grad(self, softmax):
    s = softmax.reshape(-1,1)
    return np.diagflat(s) - np.dot(s, s.T)

  def p_gradient(self, state, probs, action):
    # derivative of activation function 
    dlog = self.softmax_grad(probs)[action,:]
    grad = state.reshape(-1,1).dot(dlog.reshape(1,-1))  # geet gradient dot product of dlog and input
    grad = grad/ probs[action]
    return grad

In [0]:
env = gym.make('CartPole-v1') # observation [position of cart, velocity of cart, angle of pole, rotation rate of pole]
np.random.seed(1)
p_agent = Vanilla_reinforce_agent(env.observation_space.shape, env.action_space.n)

p_agent.train(env, 5000, 500)

Episode 0: 28.0
Episode 1: 0.0
Episode 2: 5.0
Episode 3: 16.0
Episode 4: 2.0
Episode 5: 16.0
Episode 6: -2.0
Episode 7: 16.0
Episode 8: 5.0
Episode 9: 0.0
Episode 10: 55.0
Episode 11: 16.0
Episode 12: 21.0
Episode 13: 5.0
Episode 14: 0.0
Episode 15: 4.0
Episode 16: 9.0
Episode 17: 6.0
Episode 18: 12.0
Episode 19: 10.0
Episode 20: 2.0
Episode 21: 18.0
Episode 22: 36.0
Episode 23: 2.0
Episode 24: 9.0
Episode 25: 9.0
Episode 26: 0.0
Episode 27: 2.0
Episode 28: 24.0
Episode 29: 18.0
Episode 30: 2.0
Episode 31: 3.0
Episode 32: 17.0
Episode 33: 29.0
Episode 34: 1.0
Episode 35: 30.0
Episode 36: 8.0
Episode 37: 4.0
Episode 38: 10.0
Episode 39: 7.0
Episode 40: 5.0
Episode 41: 4.0
Episode 42: 24.0
Episode 43: 22.0
Episode 44: 7.0
Episode 45: 25.0
Episode 46: 5.0
Episode 47: 8.0
Episode 48: 0.0
Episode 49: 12.0
Episode 50: 18.0
Episode 51: 3.0
Episode 52: 1.0
Episode 53: 4.0
Episode 54: 13.0
Episode 55: 3.0
Episode 56: 7.0
Episode 57: 3.0
Episode 58: 28.0
Episode 59: 5.0
Episode 60: 21.0
Episode 

In [0]:
env = utils.wrap_env(gym.make("CartPole-v1"))
observation = env.reset()
total_raward_rnd = 0
while True:
    env.render()
    policy = p_agent.policy(observation)
    action = np.random.choice(p_agent.actions, p=policy)
    observation, reward, done, info = env.step(action)
    total_raward_rnd += reward
    if done: 
      break;
print(total_raward_rnd)
env.close()
utils.show_video()

500.0


# Deep Deterministic Policy Gradients

  ![3-1](https://drive.google.com/uc?id=17EOxQZzoW3PiGzUhW-K7mvFGs3rgzfaJ)

In [0]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward', 'done'))

class ReplayMemory(object):
    '''
          A simple memory for storing episodes where each episodes 
          is a names tuple with (state, action, next_state, reward, done)
    '''

    def __init__(self, capacity):
        '''
            Initialize memory of size capacity
            Input: Capacity : int 
                        size of the memory

            output: initialized ReplayMemory object
        '''
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        '''
            input *args : list  *args is list for transition 
            [state, action, next_state, reward, done] and add
            transition to memory.
            Returns : None
        '''
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        '''
            Randomly sample transitions from memory
            Input batch_size : int
                    numer of transition to sample
            Output:  namedtuple
                      Namedtupe with each field contains a list of data points

        '''
        batch = random.sample(self.memory, batch_size)
        return Transition(*zip(*batch))


    def __len__(self):
        '''
              returns current size of memory
        '''
        return len(self.memory)

In [0]:
# [reference] https://github.com/matthiasplappert/keras-rl/blob/master/rl/random.py

class RandomProcess(object):
    def reset_states(self):
        pass

class AnnealedGaussianProcess(RandomProcess):
    def __init__(self, mu, sigma, sigma_min, n_steps_annealing):
        self.mu = mu
        self.sigma = sigma
        self.n_steps = 0

        if sigma_min is not None:
            self.m = -float(sigma - sigma_min) / float(n_steps_annealing)
            self.c = sigma
            self.sigma_min = sigma_min
        else:
            self.m = 0.
            self.c = sigma
            self.sigma_min = sigma

    @property
    def current_sigma(self):
        sigma = max(self.sigma_min, self.m * float(self.n_steps) + self.c)
        return sigma


# Based on http://math.stackexchange.com/questions/1287634/implementing-ornstein-uhlenbeck-in-matlab
class OrnsteinUhlenbeckProcess(AnnealedGaussianProcess):
    def __init__(self, theta, mu=0., sigma=1., dt=1e-2, x0=None, size=1, sigma_min=None, n_steps_annealing=1000):
        super(OrnsteinUhlenbeckProcess, self).__init__(mu=mu, sigma=sigma, sigma_min=sigma_min, n_steps_annealing=n_steps_annealing)
        self.theta = theta
        self.mu = mu
        self.dt = dt
        self.x0 = x0
        self.size = size
        self.reset_states()

    def sample(self):
        x = self.x_prev + self.theta * (self.mu - self.x_prev) * self.dt + self.current_sigma * np.sqrt(self.dt) * np.random.normal(size=self.size)
        self.x_prev = x
        self.n_steps += 1
        return x

    def reset_states(self):
        self.x_prev = self.x0 if self.x0 is not None else np.zeros(self.size)

In [0]:
class Abstract_DQNN(nn.Module):
    '''
            Abstract class gives skelleton that all DQN algos should implement/inherit

            DQNN are the neural network of the Deep Q learning part. Deep Q learning consits of netowkr and agent.
    '''
    def __init__(self, input_dim, output_dim):

        '''
            input_dim : tuple  shape of enviroment state
            output_dim: int    number of actions  
        '''
        super(Abstract_DQNN, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self._init_fc()
        self._init_o()

    def _init_fc(self):
        '''
              Initialize the feature generation part of the NN
        '''
        pass
    
    def _init_o(self):
        '''
              Initialize the output layer of the NN
        '''
        pass
    
    def _forward_fc(self, state):
        '''
              Pass the state trough the feature layer generating hidden state (i.e. features)
        '''
        pass

    def _forward_o(self, fc):
        '''
              pass features from fc layer trough the output layer
        '''
        pass

    def forward(self, state):
        '''
            Complete forward pass from state to otput
            Input: State: list   state of the enviroment
            Output: Value function for each action in input state

        '''
        features = self._forward_fc(state)
        features = features.view(features.size(0), -1)
        out = self._forward_o(features)
        return out

    def init_weights_for_linear(self, m):
        if type(m) == nn.Linear:
            fanin = m.weight.data.size()
            v = 1. / np.sqrt(fanin)
            m.weight.data =  torch.Tensor(size).uniform_(-v, v)


class Actor(Abstract_DQNN):
    '''
          Fully connected Nural network for Q-Learning
    '''
    def _init_fc(self):
        '''
              Initialize the feature generation part of the NN
              Here it is two dense layer
        '''
        self.fc = nn.Sequential(
            nn.Linear(self.input_dim[0], 4),
            nn.ReLU(),
            nn.Linear(4, 4),
            nn.ReLU(),
        )

    def _init_o(self):
        '''
              pass features from fc layer trough the output layer
              here a single dense layer
        '''
        self.o = nn.Sequential(
            nn.Linear(4, self.output_dim),
            nn.Tanh()
        )

    def _forward_fc(self, state):
        '''
              Pass the state trough the feature layer generating hidden state (i.e. features)
              Input state env.state state of the enviroment
              returns hidden state of fc layers
        '''
        return self.fc(state)

    def _forward_o(self, fcs):
        '''
              pass features from fc layer trough the output layer
              Input output hidden layer of fc layer
              Output value state action function for each action in state
        '''
        return self.o(fcs)


class Critic(Abstract_DQNN):


    '''
        Fully connected Nural network for Q-Learning
    '''
    
    def _init_fc(self):
        '''
              Initialize the feature generation part of the NN
              Here it is two dense layer
        '''
        self.fc1 = nn.Sequential(
            nn.Linear(self.input_dim[0], 4),
            nn.ReLU()
        )
        
         

        self.fc2 = nn.Sequential(
            nn.Linear(4+self.output_dim, 4),
            nn.ReLU(),
            nn.Linear(4, 4),
            nn.ReLU(),
        )

    def _init_o(self):
        '''
              pass features from fc layer trough the output layer
              here a single dense layer
        '''
        self.o = nn.Sequential(
            nn.Linear(4, 1),
        )

    def _forward_fc(self, state):
        '''
              Pass the state trough the feature layer generating hidden state (i.e. features)
              Input state env.state state of the enviroment
              returns hidden state of fc layers
        '''
        (state, a) = state
        h1 = self.fc1(state)
        out =  self.fc2(torch.cat([h1,a],1))
        return out

    def _forward_o(self, fcs):
        '''
              pass features from fc layer trough the output layer
              Input output hidden layer of fc layer
              Output value state action function for each action in state
        '''
        return self.o(fcs)



In [0]:
class DDPG_Agent:

    '''
          Base model for Deep Q-Learning Agents
    '''

    def __init__(self, env, replay_buffer, nb_actions,
                 learning_rate=0.001, 
                 gamma=0.99, 
                 update_target = 100, 
                 eps=1, 
                 eps_decay=0.01,  
                 tau = 0.001, 
                 ou_theta = 0.15,
                 ou_mu = 0.0,
                 ou_sigma = 0.2):
        '''
              Iniitialize Deep Q Agent

              Input:

                        env: openai gym enviroment
                        NN_class: Abstract_DQNN Neural network class for neural network part of Deep Q-Learning
                        replay_buffer: Replay Bufer to store Transitions
                        learning_rate: learning rate to update neural network
                        gamma: future discount rate
                        update_target: step % update_target ==0 is when target network gets updated
                        eps: epsilon for choosing random action
                        eps_decay: rate at which epsilon decays
                        eps_min: minimal epsilon value 
        '''
        self.nb_actions = nb_actions
        self.nb_states  = env.observation_space.shape
        self.env = env
        self.tau = tau
        self.learning_rate = learning_rate
        self.gamma = gamma
        self.replay_buffer = replay_buffer
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.update_target = update_target
        self.i = 0
        self.eps = eps
        self.eps_decay = eps_decay
        self.random_process = OrnsteinUhlenbeckProcess(size=self.nb_actions, theta=ou_theta, mu=ou_mu, sigma=ou_sigma)


        self.actor = Actor(self.nb_states, self.nb_actions)
        self.actor_target = Actor(self.nb_states, self.nb_actions)
        self.actor_optim  = torch.optim.Adam(self.actor.parameters(), lr=learning_rate)

        self.critic = Critic(self.nb_states, self.nb_actions)
        self.critic_target = Critic(self.nb_states, self.nb_actions)
        self.critic_optim  = torch.optim.Adam(self.critic.parameters(), lr=learning_rate)

######################################################################
############################  Core Functons ##########################
######################################################################


    def random_action(self):
        action = np.random.uniform(-1.,1.,self.nb_actions)
        return action

    def get_action(self, state, is_training=1):
        state = torch.FloatTensor(state).float().unsqueeze(0).to(self.device)
        action = self.actor.forward(state).squeeze(0).data.numpy()
        action += is_training*max(self.eps, 0)*self.random_process.sample()
        action = np.clip(action, -1., 1.)        
        return action

    def train(self, env, max_episodes, max_steps, batch_size, warm_up=100):
        '''
                trains Deep Q Agent for a given enviroment
                Input:
                        env: enviroment to train on
                        max_episodes: number of epsiodes to play
                        max_steps: number of steps to take in an epsiode
                        batch_size: size of batch
        '''
        episode_rewards = []

        for episode in range(max_episodes):
            state = env.reset()
            episode_reward = 0

            for step in range(max_steps):
                if warm_up > 0:
                  action = self.random_action()
                  warm_up -= 1
                else:
                  action = self.get_action(state)

                next_state, reward, done, _ = env.step(action)
                self.replay_buffer.push(state, action, next_state, reward, done)
                episode_reward += reward

                if len(self.replay_buffer) > batch_size:
                    self.update(batch_size)   

                if done or step == max_steps-1:
                    episode_rewards.append(episode_reward)
                    self.update_eps()
                    print("Episode " + str(episode) + ": " + str(episode_reward), '\t', self.eps)
                    break

                state = next_state

        return episode_rewards

    def update(self, batch_size):

        '''
              Forward and backward pass for a batch
                Input: 
                        batch_size: number of Transitions to be sampled from memory
        '''
        batch = self.replay_buffer.sample(batch_size)
        states, actions, rewards, next_states, dones = self.__get_tensors_from_batch__(batch)

        actor_policies = self.forward_actor(next_states)
        actor_policies = actor_policies.view(actor_policies.size(0), 1)
        
        next_q_values = self.forward_critic(states, actor_policies)
        next_q_values = next_q_values.view(next_q_values.size(0), 1)

        yis = self._yis(rewards, dones, next_q_values)

        self.update_critic(states, actions, yis)
        self.update_actor(states)


        # Copy the moving NN in the target NN
        if self.i % self.update_target == 0:
          self.soft_update(self.actor_target, self.actor)
          self.soft_update(self.critic_target, self.critic)
        self.i += 1

    def forward_critic(self, next_states, actor_policies):
        return self.critic_target([next_states, actor_policies]) # run forward critic form state, policy -> q-value


    def update_critic(self, states, actions, yis):
        q_batch = self.critic([ states, actions ]) # get q_vals by running critic forward critic
        loss_critic = F.mse_loss(q_batch, yis)     # get  F.mse_loss betwen yis and q-vals
        self.critic.zero_grad()
        loss_critic.backward()
        self.critic_optim.step()

    def forward_actor(self, next_state):
        return self.actor_target(next_state) # run forward actor form state -> policy

    def update_actor(self, states):
        
        policies = self.actor.forward(states)  # forwad actor
        policies = policies.view(policies.size(0), 1)  
        actor_loss = self.critic([  # forward critic
              states,
              policies
          ])

        actor_loss = (-actor_loss).mean()
        self.actor.zero_grad()
        actor_loss.backward()
        self.actor_optim.step()


    def _yis(self, rewards, dones, next_Q):
      return rewards + (1 - dones) * self.gamma * next_Q # calculate yis = reward + (1 - done)*gamma*next_Q    

######################################################################
############################  Helper Functons ########################
######################################################################

    def __get_tensors_from_batch__(self, batch):
        '''
              maps list values from abtch to pytorch tensors

              Input: batch: list of Transitions

              output: pytorch tensors of all Transitions values
        '''
        states = torch.FloatTensor(batch.state, device=self.device)
        actions = torch.FloatTensor(batch.action, device=self.device)
        rewards = torch.FloatTensor(batch.reward, device=self.device)
        next_states = torch.FloatTensor(batch.next_state, device=self.device)
        dones = torch.FloatTensor(batch.done, device=self.device)
        
        # resize tensors
        actions = actions.view(actions.size(0), 1)
        dones = dones.view(dones.size(0), 1)
        rewards = rewards.view(rewards.size(0), 1)

        return states, actions, rewards, next_states, dones

    def update_eps(self):
      '''
              perform epsilon decay
      '''
      if self.eps > 0:
          self.eps -= self.eps_decay

    def soft_update(self, target, source):
      for target_param, param in zip(target.parameters(), source.parameters()):
        target_param.data.copy_(target_param.data * (1.0 - self.tau) + param.data * self.tau)


In [0]:
env_id = "Pendulum-v0"
MAX_EPISODES = 50
MAX_STEPS = 500
BATCH_SIZE = 32

env = gym.make(env_id)
buffer = ReplayMemory(10000)

agent = DDPG_Agent(env, buffer, 1)
agent.train(env, 60000, 200, 32)

Output hidden; open in https://colab.research.google.com to view.

# Build your own open AI enviroment
## Custom enviroments

In [0]:
import gym
from gym import spaces

class CustomEnv(gym.Env):
  """Custom Environment that follows gym interface"""
  metadata = {'render.modes': ['human']}

  def __init__(self, arg1, arg2, ...):
    super(CustomEnv, self).__init__()    # Define action and observation space
    # They must be gym.spaces objects    # Example when using discrete actions:
    self.action_space = spaces.Discrete(N_DISCRETE_ACTIONS)    # Example for using image as input:
    self.observation_space = spaces.Box(low=0, high=255, shape=
                    (HEIGHT, WIDTH, N_CHANNELS), dtype=np.uint8)

  def _step(self, action):
        """

        Parameters
        ----------
        action :

        Returns
        -------
        ob, reward, episode_over, info : tuple
            ob (object) :
                an environment-specific object representing your observation of
                the environment.
            reward (float) :
                amount of reward achieved by the previous action. The scale
                varies between environments, but the goal is always to increase
                your total reward.
            episode_over (bool) :
                whether it's time to reset the environment again. Most (but not
                all) tasks are divided up into well-defined episodes, and done
                being True indicates the episode has terminated. (For example,
                perhaps the pole tipped too far, or you lost your last life.)
            info (dict) :
                 diagnostic information useful for debugging. It can sometimes
                 be useful for learning (for example, it might contain the raw
                 probabilities behind the environment's last state change).
                 However, official evaluations of your agent are not allowed to
                 use this for learning.
        """
        self._take_action(action)
        self.status = self.env.step()
        reward = self._get_reward()
        ob = self.env.getState()
        episode_over = self.status != hfo_py.IN_GAME
        return ob, reward, episode_over, {}


        def _reset(self):
        pass

    def _render(self, mode='human', close=False):
        pass

    def _take_action(self, action):
        pass

    def _get_reward(self):
        """ Reward is given for XY. """
        if self.status == FOOBAR:
            return 1
        elif self.status == ABC:
            return self.somestate ** 2
        else:
            return 0

In [0]:
import sys
from contextlib import closing

import numpy as np
from six import StringIO, b

from gym import utils
from gym.envs.toy_text import discrete

LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3

MAPS = {
    "4x4": [
        "SFFF",
        "FHFH",
        "FFFH",
        "HFFG"
    ],
    "8x8": [
        "SFFFFFFF",
        "FFFFFFFF",
        "FFFHFFFF",
        "FFFFFHFF",
        "FFFHFFFF",
        "FHHFFFHF",
        "FHFFHFHF",
        "FFFHFFFG"
    ],
}


def generate_random_map(size=8, p=0.8):
    """Generates a random valid map (one that has a path from start to goal)
    :param size: size of each side of the grid
    :param p: probability that a tile is frozen
    """
    valid = False

    # DFS to check that it's a valid path.
    def is_valid(res):
        frontier, discovered = [], set()
        frontier.append((0,0))
        while frontier:
            r, c = frontier.pop()
            if not (r,c) in discovered:
                discovered.add((r,c))
                directions = [(1, 0), (0, 1), (-1, 0), (0, -1)]
                for x, y in directions:
                    r_new = r + x
                    c_new = c + y
                    if r_new < 0 or r_new >= size or c_new < 0 or c_new >= size:
                        continue
                    if res[r_new][c_new] == 'G':
                        return True
                    if (res[r_new][c_new] not in '#H'):
                        frontier.append((r_new, c_new))
        return False

    while not valid:
        p = min(1, p)
        res = np.random.choice(['F', 'H'], (size, size), p=[p, 1-p])
        res[0][0] = 'S'
        res[-1][-1] = 'G'
        valid = is_valid(res)
    return ["".join(x) for x in res]


class FrozenLakeEnv(discrete.DiscreteEnv):
    """
    Winter is here. You and your friends were tossing around a frisbee at the park
    when you made a wild throw that left the frisbee out in the middle of the lake.
    The water is mostly frozen, but there are a few holes where the ice has melted.
    If you step into one of those holes, you'll fall into the freezing water.
    At this time, there's an international frisbee shortage, so it's absolutely imperative that
    you navigate across the lake and retrieve the disc.
    However, the ice is slippery, so you won't always move in the direction you intend.
    The surface is described using a grid like the following
        SFFF
        FHFH
        FFFH
        HFFG
    S : starting point, safe
    F : frozen surface, safe
    H : hole, fall to your doom
    G : goal, where the frisbee is located
    The episode ends when you reach the goal or fall in a hole.
    You receive a reward of 1 if you reach the goal, and zero otherwise.
    """

    metadata = {'render.modes': ['human', 'ansi']}

    def __init__(self, desc=None, map_name="4x4",is_slippery=True):
        if desc is None and map_name is None:
            desc = generate_random_map()
        elif desc is None:
            desc = MAPS[map_name]
        self.desc = desc = np.asarray(desc,dtype='c')
        self.nrow, self.ncol = nrow, ncol = desc.shape
        self.reward_range = (0, 1)

        nA = 4
        nS = nrow * ncol

        isd = np.array(desc == b'S').astype('float64').ravel()
        isd /= isd.sum()

        P = {s : {a : [] for a in range(nA)} for s in range(nS)}

        def to_s(row, col):
            return row*ncol + col

        def inc(row, col, a):
            if a == LEFT:
                col = max(col-1,0)
            elif a == DOWN:
                row = min(row+1,nrow-1)
            elif a == RIGHT:
                col = min(col+1,ncol-1)
            elif a == UP:
                row = max(row-1,0)
            return (row, col)

        for row in range(nrow):
            for col in range(ncol):
                s = to_s(row, col)
                for a in range(4):
                    li = P[s][a]
                    letter = desc[row, col]
                    if letter in b'GH':
                        li.append((1.0, s, 0, True))
                    else:
                        if is_slippery:
                            for b in [(a-1)%4, a, (a+1)%4]:
                                newrow, newcol = inc(row, col, b)
                                newstate = to_s(newrow, newcol)
                                newletter = desc[newrow, newcol]
                                done = bytes(newletter) in b'GH'
                                rew = float(newletter == b'G')
                                li.append((1.0/3.0, newstate, rew, done))
                        else:
                            newrow, newcol = inc(row, col, a)
                            newstate = to_s(newrow, newcol)
                            newletter = desc[newrow, newcol]
                            done = bytes(newletter) in b'GH'
                            rew = float(newletter == b'G')
                            li.append((1.0, newstate, rew, done))

        super(FrozenLakeEnv, self).__init__(nS, nA, P, isd)

    def render(self, mode='human'):
        outfile = StringIO() if mode == 'ansi' else sys.stdout

        row, col = self.s // self.ncol, self.s % self.ncol
        desc = self.desc.tolist()
        desc = [[c.decode('utf-8') for c in line] for line in desc]
        desc[row][col] = utils.colorize(desc[row][col], "red", highlight=True)
        if self.lastaction is not None:
            outfile.write("  ({})\n".format(["Left","Down","Right","Up"][self.lastaction]))
        else:
            outfile.write("\n")
        outfile.write("\n".join(''.join(line) for line in desc)+"\n")

        if mode != 'human':
            with closing(outfile):
                return outfile.getvalue()

## Wrappers

https://github.com/openai/baselines/blob/master/baselines/common/atari_wrappers.py

In [0]:
class FrameStack(gym.Wrapper):
    def __init__(self, env, k):
        """Stack k last frames.
        Returns lazy array, which is much more memory efficient.
        See Also
        --------
        baselines.common.atari_wrappers.LazyFrames
        """
        gym.Wrapper.__init__(self, env)
        self.k = k
        self.frames = deque([], maxlen=k)
        shp = env.observation_space.shape
        self.observation_space = spaces.Box(low=0, high=255, shape=(shp[:-1] + (shp[-1] * k,)), dtype=env.observation_space.dtype)

    def reset(self):
        ob = self.env.reset()
        for _ in range(self.k):
            self.frames.append(ob)
        return self._get_ob()

    def step(self, action):
        ob, reward, done, info = self.env.step(action)
        self.frames.append(ob)
        return self._get_ob(), reward, done, info

    def _get_ob(self):
        assert len(self.frames) == self.k
        return LazyFrames(list(self.frames))




In [0]:
class ClipRewardEnv(gym.RewardWrapper):
    def __init__(self, env):
        gym.RewardWrapper.__init__(self, env)

    def reward(self, reward):
        """Bin reward to {+1, 0, -1} by its sign."""
        return np.sign(reward)

In [0]:
class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env, skip=4):
        """Return only every `skip`-th frame"""
        gym.Wrapper.__init__(self, env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = np.zeros((2,)+env.observation_space.shape, dtype=np.uint8)
        self._skip       = skip

    def step(self, action):
        """Repeat action, sum reward, and max over last observations."""
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            if i == self._skip - 2: self._obs_buffer[0] = obs
            if i == self._skip - 1: self._obs_buffer[1] = obs
            total_reward += reward
            if done:
                break
        # Note that the observation on the done=True frame
        # doesn't matter
        max_frame = self._obs_buffer.max(axis=0)

        return max_frame, total_reward, done, info

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)